## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1, l2, l1_l2

"""
建立神經網路，並加入 L1 或 L2
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l2_ratio=1e-4):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l2(l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l2(l2_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
LEARNING_RATE = 1e-3
EPOCHS = 40
BATCH_SIZE = 256
MOMENTUM = 0.95
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12]

In [ ]:
results = {}
"""
使用迴圈建立不同的帶不同 L1/L2 的模型並訓練
"""
for regulizer_ratio in L2_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp(input_shape=x_train.shape[1:], l2_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l2-%s" % str(regulizer_ratio)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


Experiment with Regulizer = 0.010000
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Instructions for updati

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 11s 215us/step - loss: 2.1943 - acc: 0.2649 - val_loss: 2.0354 - val_acc: 0.3344
Epoch 2/40
50000/50000 [==============================] - 10s 195us/step - loss: 1.9667 - acc: 0.3596 - val_loss: 1.9147 - val_acc: 0.3773
Epoch 3/40
50000/50000 [==============================] - 10s 196us/step - loss: 1.8768 - acc: 0.3902 - val_loss: 1.8460 - val_acc: 0.4000
Epoch 4/40
50000/50000 [==============================] - 10s 197us/step - loss: 1.8153 - acc: 0.4137 - val_loss: 1.8038 - val_acc: 0.4125
Epoch 5/40
50000/50000 [==============================] - 12s 236us/step - loss: 1.7660 - acc: 0.4305 - val_loss: 1.7487 - val_acc: 0.4356
Epoch 6/40
50000/50000 [==============================] - 13s 254us/step - loss: 1.7267 - acc: 0.4446 - val_loss: 1.7234 - val_acc: 0.4485
Epoch 7/40
50000/50000 [==============================] - 10s 204us/step - loss: 1.6925 - acc: 0.4557 - val_loss: 1.

Epoch 12/40
50000/50000 [==============================] - 7s 146us/step - loss: 1.4133 - acc: 0.5034 - val_loss: 1.4755 - val_acc: 0.4735
Epoch 13/40
50000/50000 [==============================] - 7s 145us/step - loss: 1.3911 - acc: 0.5115 - val_loss: 1.4328 - val_acc: 0.4908
Epoch 14/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.3686 - acc: 0.5187 - val_loss: 1.4197 - val_acc: 0.4956
Epoch 15/40
50000/50000 [==============================] - 7s 148us/step - loss: 1.3503 - acc: 0.5264 - val_loss: 1.4230 - val_acc: 0.4922
Epoch 16/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.3300 - acc: 0.5321 - val_loss: 1.4029 - val_acc: 0.4949
Epoch 17/40
50000/50000 [==============================] - 8s 164us/step - loss: 1.3117 - acc: 0.5397 - val_loss: 1.4307 - val_acc: 0.4908
Epoch 18/40
50000/50000 [==============================] - 8s 160us/step - loss: 1.2940 - acc: 0.5452 - val_loss: 1.4002 - val_acc: 0.4977
Epoch 19/40
50000/50000 [==

50000/50000 [==============================] - 9s 174us/step - loss: 1.2091 - acc: 0.5753 - val_loss: 1.3863 - val_acc: 0.5048
Epoch 24/40
50000/50000 [==============================] - 8s 170us/step - loss: 1.1938 - acc: 0.5804 - val_loss: 1.3793 - val_acc: 0.5109
Epoch 25/40
50000/50000 [==============================] - 8s 156us/step - loss: 1.1770 - acc: 0.5859 - val_loss: 1.3725 - val_acc: 0.5153
Epoch 26/40
50000/50000 [==============================] - 8s 156us/step - loss: 1.1661 - acc: 0.5890 - val_loss: 1.3344 - val_acc: 0.5273
Epoch 27/40
50000/50000 [==============================] - 8s 162us/step - loss: 1.1497 - acc: 0.5945 - val_loss: 1.3549 - val_acc: 0.5258
Epoch 28/40
50000/50000 [==============================] - 7s 149us/step - loss: 1.1372 - acc: 0.6006 - val_loss: 1.3599 - val_acc: 0.5217
Epoch 29/40
50000/50000 [==============================] - 8s 159us/step - loss: 1.1218 - acc: 0.6042 - val_loss: 1.3818 - val_acc: 0.5121
Epoch 30/40
50000/50000 [==============

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()